In [3]:
%load_ext autoreload
%autoreload 2

import dataclasses

from src.networks.fnn import *
from src.networks.self_normalizing_fnn import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
FNN.from_hyper_parameters(SelfNormalizingFNNHyperParameters(1, [2], 3, activate_last_layer=True))

FNN(
  (fnn): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): SELU()
    (2): Linear(in_features=2, out_features=3, bias=True)
    (3): SELU()
  )
)